In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Outros/Colab Notebooks/30 Days Of Bert/Day01/train (2).csv")
dataset['train'][100]

Generating train split: 0 examples [00:00, ? examples/s]

{'text': 'kkkkkkkk que otário tomo um pau', 'label': 1}

In [4]:
test = load_dataset("csv", data_files="/content/drive/MyDrive/Outros/Colab Notebooks/30 Days Of Bert/Day01/test (4).csv")
test['train'][10]

Generating train split: 0 examples [00:00, ? examples/s]

{'id': 10,
 'text': '@user @user @user @user @user Olha esse corpo... Essa bucetinha linda e esses seios na medida certa... Ahhh, eu com um mulherão desses em casa nunca mais veria a luz do dia! Kkkkk 😂😋❤️'}

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_train = dataset.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/16800 [00:00<?, ? examples/s]

Map:   0%|          | 0/4200 [00:00<?, ? examples/s]

In [7]:
small_train_dataset = tokenized_train["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_test["train"].shuffle(seed=42).select(range(1000))

In [8]:
small_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [9]:
small_eval_dataset

Dataset({
    features: ['id', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2, torch_dtype="auto")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch", logging_steps=1)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,

)

In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ifaledu2017 (ifaledu2017-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.634100,No log
2,0.544400,No log
3,0.503500,No log


TrainOutput(global_step=375, training_loss=0.6325824644168218, metrics={'train_runtime': 391.5498, 'train_samples_per_second': 7.662, 'train_steps_per_second': 0.958, 'total_flos': 789333166080000.0, 'train_loss': 0.6325824644168218, 'epoch': 3.0})

In [17]:
trainer.evaluate()

{'eval_runtime': 29.129,
 'eval_samples_per_second': 34.33,
 'eval_steps_per_second': 4.291,
 'epoch': 3.0}

In [18]:
trainer.save_model("/content/drive/MyDrive/Outros/Colab Notebooks/30 Days Of Bert/Day01")

In [21]:
from huggingface_hub import notebook_login

In [22]:
notebook_login()

In [23]:
trainer.push_to_hub("CASLL/Toxic-LanguageBR")

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

events.out.tfevents.1738384463.f983829071a4.2262.0:   0%|          | 0.00/84.7k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1738385021.f983829071a4.2262.1:   0%|          | 0.00/311 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CASLL/test_trainer/commit/989ba0338d5aefa689257ccd3088d18161a3cbe1', commit_message='CASLL/Toxic-LanguageBR', commit_description='', oid='989ba0338d5aefa689257ccd3088d18161a3cbe1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/CASLL/test_trainer', endpoint='https://huggingface.co', repo_type='model', repo_id='CASLL/test_trainer'), pr_revision=None, pr_num=None)

**Huh**: Why `no log` in Training and Validation Loss.

**Aha**: Maybe I nee mude the logging step: https://github.com/huggingface/transformers/issues/8910 and https://discuss.huggingface.co/t/how-to-save-my-model-to-use-it-later/20568

**Q**: How I train in the GPU?

**Q**: How to save the model?

In [27]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="CASLL/Toxic-LanguageBR")

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

OSError: Can't load tokenizer for 'CASLL/Toxic-LanguageBR'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'CASLL/Toxic-LanguageBR' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

Huh: Porque não consigo carregar meu modelo?